 https://github.com/JuliaLang/julia/blob/v0.5.0/NEWS.md

In [1]:
"""
Julia v0.5.0 Release Notes
==========================

New language features
---------------------

  * Generator expressions: `f(i) for i in 1:n` ([#4470]). This returns an iterator
    that computes the specified values on demand. This is useful for computing, e.g.
    `sum(f(i) for i in 1:n)` without creating an intermediate array of values.

  * Generators and comprehensions support filtering using `if` ([#550]) and nested
    iteration using multiple `for` keywords ([#4867]).

  * Fused broadcasting syntax: ``f.(args...)`` is equivalent to ``broadcast(f, args...)`` ([#15032]),
    and nested `f.(g.(args...))` calls are fused into a single `broadcast` loop ([#17300]).
    Similarly, the syntax `x .= ...` is equivalent to a `broadcast!(identity, x, ...)`
    call and fuses with nested "dot" calls; also, `x .+= y` and similar is now
    equivalent to `x .= x .+ y`, rather than `x = x .+ y` ([#17510]).

  * Macro expander functions are now generic, so macros can have multiple definitions
    (e.g. for different numbers of arguments, or optional arguments) ([#8846], [#9627]).
    However note that the argument types refer to the syntax tree representation, and not
    to the types of run time values.

  * Varargs functions like `foo{T}(x::T...)` may now restrict the number
    of such arguments using `foo{T,N}(x::Vararg{T,N})` ([#11242]).

  * `x ∈ X` is now a synonym for `x in X` in `for` loops and comprehensions,
    as it already was in comparisons ([#13824]).

  * The `PROGRAM_FILE` global is now available for determining the name of the running script ([#14114]).

  * The syntax `x.:sym` (e.g. `Base.:+`) is now supported, while using `x.(:sym)`
    or `x.(i)` for field access are deprecated in favor of `getfield` ([#15032]).

  * Function return type syntax `function f()::T` has been added ([#1090]). Values returned
    from a function with such a declaration will be converted to the specified type `T`.

  * Many more operators now support `.` prefixes (e.g. `.≤`) ([#17393]).  However,
    users are discouraged from overloading these, since they are mainly parsed
    in order to implement backwards compatibility with planned automatic
    broadcasting of dot operators in Julia 0.6 ([#16285]).  Explicitly qualified
    operator names like `Base.≤` should now use `Base.:≤` (prefixed by `@compat`
    if you need 0.4 compatibility via the `Compat` package).

  * User-extensible bounds check elimination is now possible with the new
    `@boundscheck` macro ([#14474]). This macro marks bounds checking code blocks,
    which the compiler may remove when encountered inside an `@inbounds` call.
"""
julia05() = 0

julia05

In [2]:
?julia05

search: JULIA_HOME



# Julia v0.5.0 Release Notes

## New language features

  * Generator expressions: `f(i) for i in 1:n` ([#4470]). This returns an iterator that computes the specified values on demand. This is useful for computing, e.g. `sum(f(i) for i in 1:n)` without creating an intermediate array of values.
  * Generators and comprehensions support filtering using `if` ([#550]) and nested iteration using multiple `for` keywords ([#4867]).
  * Fused broadcasting syntax: $f.(args...)$ is equivalent to $broadcast(f, args...)$ ([#15032]), and nested `f.(g.(args...))` calls are fused into a single `broadcast` loop ([#17300]). Similarly, the syntax `x .= ...` is equivalent to a `broadcast!(identity, x, ...)` call and fuses with nested "dot" calls; also, `x .+= y` and similar is now equivalent to `x .= x .+ y`, rather than `x = x .+ y` ([#17510]).
  * Macro expander functions are now generic, so macros can have multiple definitions (e.g. for different numbers of arguments, or optional arguments) ([#8846], [#9627]). However note that the argument types refer to the syntax tree representation, and not to the types of run time values.
  * Varargs functions like `foo{T}(x::T...)` may now restrict the number of such arguments using `foo{T,N}(x::Vararg{T,N})` ([#11242]).
  * `x ∈ X` is now a synonym for `x in X` in `for` loops and comprehensions, as it already was in comparisons ([#13824]).
  * The `PROGRAM_FILE` global is now available for determining the name of the running script ([#14114]).
  * The syntax `x.:sym` (e.g. `Base.:+`) is now supported, while using `x.(:sym)` or `x.(i)` for field access are deprecated in favor of `getfield` ([#15032]).
  * Function return type syntax `function f()::T` has been added ([#1090]). Values returned from a function with such a declaration will be converted to the specified type `T`.
  * Many more operators now support `.` prefixes (e.g. `.≤`) ([#17393]).  However, users are discouraged from overloading these, since they are mainly parsed in order to implement backwards compatibility with planned automatic broadcasting of dot operators in Julia 0.6 ([#16285]).  Explicitly qualified operator names like `Base.≤` should now use `Base.:≤` (prefixed by `@compat` if you need 0.4 compatibility via the `Compat` package).
  * User-extensible bounds check elimination is now possible with the new `@boundscheck` macro ([#14474]). This macro marks bounds checking code blocks, which the compiler may remove when encountered inside an `@inbounds` call.



### Generator expressions: f(i) for i in 1:n ([#4470]). This returns an iterator that computes the specified values on demand. This is useful for computing, e.g. sum(f(i) for i in 1:n) without creating an intermediate array of values.

* https://github.com/JuliaLang/julia/issues/4470

``` sh
~/work/julia/test $ grep \#4470 * -r
functional.jl:# generators (#4470, #14848)
```



In [3]:
using Base.Test

# generators (#4470, #14848)

@test sum(i/2 for i=1:2) == 1.5
@test collect(2i for i=2:5) == [4,6,8,10]
@test collect((i+10j for i=1:2,j=3:4)) == [31 41; 32 42]
@test collect((i+10j for i=1:2,j=3:4,k=1:1)) == reshape([31 41; 32 42], (2,2,1))

let A = collect(Base.Generator(x->2x, Real[1.5,2.5]))
    @test A == [3,5]
    @test isa(A,Vector{Float64})
end

let f(g) = (@test size(g.iter)==(2,3))
    f(i+j for i=1:2, j=3:5)
end
;

In [4]:
sum(i/2 for i=1:2)

1.5

### Generators and comprehensions support filtering using if ([#550]) and nested iteration using multiple for keywords ([#4867]).

* https://github.com/JuliaLang/julia/issues/550
* https://github.com/JuliaLang/julia/issues/4867

```sh
~/work/julia/test $ grep \#4867 * -r
functional.jl:# generators with nested loops (#4867)
```

In [5]:
# generators with nested loops (#4867)

@test [(i,j) for i=1:3 for j=1:i] == [(1,1), (2,1), (2,2), (3,1), (3,2), (3,3)]

@test [(i,j) for i=1:3 for j=1:i if j>1] == [(2,2), (3,2), (3,3)]

Test Passed
  Expression: [(i,j) for i = 1:3 for j = 1:i if j > 1] == [(2,2),(3,2),(3,3)]
   Evaluated: Tuple{Int64,Int64}[(2,2),(3,2),(3,3)] == Tuple{Int64,Int64}[(2,2),(3,2),(3,3)]

In [6]:
[(i,j) for i=1:3 for j=1:i]

6-element Array{Tuple{Int64,Int64},1}:
 (1,1)
 (2,1)
 (2,2)
 (3,1)
 (3,2)
 (3,3)

### Fused broadcasting syntax:  f.(args...)f.(args...)  is equivalent to  broadcast(f,args...)broadcast(f,args...)  ([#15032]), and nested f.(g.(args...)) calls are fused into a single broadcast loop ([#17300]). Similarly, the syntax x .= ... is equivalent to a broadcast!(identity, x, ...) call and fuses with nested "dot" calls; also, x .+= y and similar is now equivalent to x .= x .+ y, rather than x = x .+ y ([#17510]).

* https://github.com/JuliaLang/julia/pull/15032
* https://github.com/JuliaLang/julia/pull/17300
* https://github.com/JuliaLang/julia/pull/17510

```sh
~/work/julia/test $ grep \#15032 * -r
broadcast.jl:# f.(args...) syntax (#15032)
core.jl:# f.(x) vectorization syntax (#15032)
```

In [7]:
# f.(args...) syntax (#15032)
let x = [1,3.2,4.7], y = [3.5, pi, 1e-4], α = 0.2342
    @test sin.(x) == broadcast(sin, x)
    @test sin.(α) == broadcast(sin, α)
    @test sin.(3.2) == broadcast(sin, 3.2) == sin(3.2)
    @test factorial.(3) == broadcast(factorial, 3)
    @test atan2.(x, y) == broadcast(atan2, x, y)
    @test atan2.(x, y') == broadcast(atan2, x, y')
    @test atan2.(x, α) == broadcast(atan2, x, α)
    @test atan2.(α, y') == broadcast(atan2, α, y')
end

Test Passed
  Expression: atan2.(α,y') == broadcast(atan2,α,y')
   Evaluated: [0.0668147 0.0744105 1.57037] == [0.0668147 0.0744105 1.57037]

In [8]:
x = [1,3.2,4.7]
sin.(x)

3-element Array{Float64,1}:
  0.841471 
 -0.0583741
 -0.999923 

### Macro expander functions are now generic, so macros can have multiple definitions (e.g. for different numbers of arguments, or optional arguments) ([#8846], [#9627]). However note that the argument types refer to the syntax tree representation, and not to the types of run time values.

* https://github.com/JuliaLang/julia/issues/8846
* https://github.com/JuliaLang/julia/issues/9627

```sh
~/work/julia/test $ grep \#8846 * -r
core.jl:# issue #8846, generic macros
```

In [9]:
# issue #8846, generic macros
macro m8846(a, b=0)
    a, b
end
@test @m8846(a) === (:a, 0)
@test @m8846(a,1) === (:a, 1)
@test_throws MethodError eval(:(@m8846(a,b,c)))

Test Passed
  Expression: eval($(Expr(:quote, :(@m8846 a b c))))
      Thrown: MethodError

In [10]:
@m8846(a)

(:a,0)

In [11]:
@m8846(a, 1)

(:a,1)

### Varargs functions like `foo{T}(x::T...)` may now restrict the number of such arguments using `foo{T,N}(x::Vararg{T,N})` ([#11242]).

* https://github.com/JuliaLang/julia/issues/11242
* http://docs.julialang.org/en/release-0.5/devdocs/types/#typevars

```sh
~/work/julia/test $ grep \#11242 * -r
abstractarray.jl:# Until #11242 is merged, we need to define each dimension independently
```

In [12]:
T,N = Array.parameters

svec(T,N)

In [13]:
dump(T)

TypeVar
  name: Symbol T
  lb: Union{}
  ub: Any
  bound: Bool false


In [14]:
foo{T,N}(x::Vararg{T,N}) = (T,N)

foo (generic function with 1 method)

In [15]:
foo("a","b")

(String,2)

In [16]:
foo(1,2,3,4,5)

(Int64,5)

### x ∈ X is now a synonym for x in X in for loops and comprehensions, as it already was in comparisons ([#13824]).

* https://github.com/JuliaLang/julia/issues/13824

* https://en.wikipedia.org/wiki/Element_(mathematics)

x ∈ A
 : x is an element of A

In [17]:
for x ∈ [1,2,3,4,5]
    println("x ", x)
end

x 1
x 2
x 3
x 4
x 5


In [18]:
A = [1,2,3,4,5]
x = 2
x ∈ A

true

### The `PROGRAM_FILE` global is now available for determining the name of the running script ([#14114]).

* https://github.com/JuliaLang/julia/issues/14114

In [19]:
PROGRAM_FILE

"/Users/wookyoung/.julia/v0.5/IJulia/src/kernel.jl"

### The syntax `x.:sym` (e.g. `Base.:+`) is now supported, while using `x.(:sym)` or `x.(i)` for field access are deprecated in favor of `getfield` ([#15032]).

* https://github.com/JuliaLang/julia/issues/15032

In [20]:
Base.:+

+ (generic function with 163 methods)

In [21]:
getfield(Base, :+)

+ (generic function with 163 methods)

###   * Function return type syntax `function f()::T` has been added ([#1090]). Values returned from a function with such a declaration will be converted to the specified type `T`.

* https://github.com/JuliaLang/julia/issues/1090

```sh
~/work/julia/test $ grep \#1090 * -r
core.jl:# issue #1090
parse.jl:# issue #10900
parse.jl:# issue #10901
```

In [22]:
# issue #1090
function f1090(x)::Int
    if x == 1
        return 1
    end
    2.0
end
@test f1090(1) === 1
@test f1090(2) === 2
g1090{T}(x::T)::T = x+1.0
@test g1090(1) === 2
@test g1090(Float32(3)) === Float32(4)

Test Passed
  Expression: g1090(Float32(3)) === Float32(4)
   Evaluated: 4.0f0 is 4.0f0

### Many more operators now support . prefixes (e.g. .≤) ([#17393]). However, users are discouraged from overloading these, since they are mainly parsed in order to implement backwards compatibility with planned automatic broadcasting of dot operators in Julia 0.6 ([#16285]). Explicitly qualified operator names like Base.≤ should now use Base.:≤ (prefixed by @compat if you need 0.4 compatibility via the Compat package).

* https://github.com/JuliaLang/julia/issues/17393
* Vectorization Roadmap #16285 https://github.com/JuliaLang/julia/issues/16285

In [23]:
≤

<= (generic function with 50 methods)

In [24]:
1 ≤ 2

true

### User-extensible bounds check elimination is now possible with the new `@boundscheck` macro ([#14474]). This macro marks bounds checking code blocks, which the compiler may remove when encountered inside an `@inbounds` call.

* https://github.com/JuliaLang/julia/pull/14474
* https://github.com/JuliaLang/julia/issues/7799

* http://docs.julialang.org/en/release-0.5/devdocs/boundscheck/

* https://en.wikipedia.org/wiki/Bounds_checking
* https://en.wikipedia.org/wiki/Inline_function

In [25]:
function inb(a)
    i = 2
    @inbounds a[i] = 5
    a
end

inb (generic function with 1 method)

In [26]:
inb([1])

1-element Array{Int64,1}:
 1

In [27]:
inb([1,2])

2-element Array{Int64,1}:
 1
 5

In [28]:
function bou(a)
    @boundscheck length(a) > 1 || throw(BoundsError("> 1"))
    a
end

bou (generic function with 1 method)

In [31]:
bou([5])

LoadError: BoundsError: attempt to access "> 1"

In [32]:
bou([5,6])

2-element Array{Int64,1}:
 5
 6

In [33]:
?checkbounds

search: checkbounds



```
checkbounds(Bool, A, I...)
```

Return `true` if the specified indices `I` are in bounds for the given array `A`. Subtypes of `AbstractArray` should specialize this method if they need to provide custom bounds checking behaviors; however, in many cases one can rely on `A`'s indices and `checkindex`.

See also `checkindex`.

```
checkbounds(A, I...)
```

Throw an error if the specified indices `I` are not in bounds for the given array `A`.


In [34]:
?checkindex

search: checkindex



```
checkindex(Bool, inds::AbstractUnitRange, index)
```

Return `true` if the given `index` is within the bounds of `inds`. Custom types that would like to behave as indices for all arrays can extend this method in order to provide a specialized bounds checking implementation.
